In [1]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
!pip install feedparser
!pip install newspaper3k

--2023-02-13 14:03:56--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8276509 (7,9M) [application/json]
Saving to: ‘lesechos.json’

lesechos.json       100%[===================>]   7,89M  11,3MB/s    in 0,7s    

2023-02-13 14:03:57 (11,3 MB/s) - ‘lesechos.json’ saved [8276509/8276509]

--2023-02-13 14:03:57--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347 [application/json]
Saving to: ‘sources.json’

sources.json        100%[===================>]     347  --.-KB/s    in 0s      

2023-02-13 14:03:57 (42,9 MB/s) - ‘sources.json’

In [2]:
!python3 -m spacy download fr_core_news_md

     |████████████████████████████████| 45.8 MB 40.9 MB/s eta 0:00:01    |████████████████▌               | 23.6 MB 6.2 MB/s eta 0:00:04
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [3]:
import spacy
nlp = spacy.load('fr_core_news_md')

In [5]:
from urllib.parse import urlparse

import json
f = open('lesechos.json')
data_lesechos = json.load(f)

# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data = fp.parse(url)

for item in data.entries:
    print(item.title, item.published, item.link)

f = open('sources.json')
liste_rss_flux = json.load(f)
liste_rss_flux = list(liste_rss_flux.values())
liste_rss_flux_filtered = [urlparse(temp[0]).path[:-12] for temp in liste_rss_flux]

liste_new_articles = []
liste_url_article_already_in_base = list(data_lesechos.keys())
for url in liste_rss_flux:
    temp_list_article = []
    data_rss = fp.parse(url)
    for item in data_rss.entries:
        link = item.link
        

    

Réforme des retraites : les syndicats haussent le ton et prédisent une « journée noire » le 7 mars Mon, 13 Feb 2023 10:25:42 +0100 https://www.lemonde.fr/politique/article/2023/02/13/reforme-des-retraites-les-syndicats-haussent-le-ton-et-predisent-une-journee-noire-le-7-mars_6161625_823448.html
Séisme en Turquie et en Syrie : plus de 35 000 morts et 400 000 personnes évacuées de la région Mon, 13 Feb 2023 14:20:10 +0100 https://www.lemonde.fr/international/article/2023/02/13/seisme-en-turquie-et-en-syrie-le-bilan-toujours-provisoire-s-eleve-a-plus-de-35-000-morts_6161652_3210.html
Alexeï Venediktov : « La haine est entrée au sein de chaque famille russe » Mon, 13 Feb 2023 06:00:49 +0100 https://www.lemonde.fr/international/article/2023/02/13/alexei-venediktov-la-haine-est-entree-au-sein-de-chaque-famille-russe_6161602_3210.html
En Australie, l’alcool de nouveau interdit dans les communautés aborigènes du Territoire du Nord Mon, 13 Feb 2023 03:42:49 +0100 https://www.lemonde.fr/internat

In [6]:
for value in data_lesechos.values():
    print(value['title'])

Economie : L'UE ébauche un plan pour qualifier le gaz et le nucléaire d'investissements "verts"
Economie : Le gouvernement veut éviter les pénuries de main-d'oeuvre dans les entreprises
Analyse et Stratégie : 2022 fera la part belle au pricing power en Bourse
Analyse et Stratégie : Les leçons à tirer de 2021 en Bourse
Marchés américains : Wall Street termine l'année près de ses records
Rétro de la semaine : Paris et New York au zénith
A ne pas manquer demain : Les indices PMI manufacturiers pour commencer l'année boursière
Paris Clôture : Malgré un léger repli en fin de parcours, le Cac 40 boucle sa meilleure année en plus de 20 ans
Matières premières : Pétrole Brent : Enquête :  :  -Omicron, nouvelle menace pour les cours du pétrole
Or : L'once d'or poursuit sa progression vendredi
Paris : Malgré son léger repli, le Cac 40 est en passe de boucler une année faste
Marchés européens : ANALYSE TECHNIQUE: Le contrat sur le CAC 40 est orienté à la baisse
Paris Ouverture : Le Cac 40 n'a plus

In [7]:
temp = list(data_lesechos.keys())

In [8]:
from urllib.parse import urlparse

temp_one = 'https://investir.lesechos.fr/investir-responsable/environnement/faire-mieux-en-consommant-moins-axe-central-de-la-transition-1900752'
res = urlparse(temp_one)
res.path[:-8]

'/investir-responsable/environnement/faire-mieux-en-consommant-moins-axe-central-de-la-transition'